In [33]:
import pyomo.environ as pyo
from typing import Union
import polars as pl

In [2]:

model = pyo.AbstractModel()

model.I = pyo.Set()

model.L= pyo.Set()
model.S = pyo.Set(within=model.L)
model.F= pyo.Set(model.L, within=model.I *model.I)

model.LF = pyo.Set(
    dimen=3, 
    initialize=lambda model: [(l, f) for l in model.L for f in model.F[l]]
)

model.nS = pyo.Set(
    dimen=1, 
    initialize=lambda model: [l for l in model.L if l not in model.S]
)

model.X = pyo.Param(model.L, default=0.0)
model.Z = pyo.Param(model.I, default=20.0)
model.P = pyo.Var(model.LF, domain=pyo.NonNegativeReals)

  
def test_propagation(model, l):
    left = model.F[l].first()
    right = model.F[l].last()
    return model.P[l, left] + model.P[l, right] == model.X[l]

def test_propagation_2(model, l):
    left = model.F[l].first()

    return model.P[l, left]  ==  10
    
def test_objective(model):
    
    return 10

model.objective = pyo.Objective(rule=test_objective, sense=pyo.maximize)
model.test_propagation = pyo.Constraint(model.L, rule=test_propagation)
model.test_propagation_2 = pyo.Constraint(model.L, rule=test_propagation_2)



In [4]:
data = {
    None: {
        'I': {None: list(range(4))},
        "L": {None: list(range(3))},
        "F": {0: [(1, 0), (0, 1)], 1: [(2, 1), (1, 2)], 2: [(3, 2), (2, 3)]},
        "S": {None: list(range(1))},
        "X": {
            0: 10,
            1: 35,
            2: 25,
        },
        'Z': {0 : 100.0, 1: 50.0, 2: 200.0}
    }
}

instance: pyo.Model = model.create_instance(data)

solver = pyo.SolverFactory('gurobi')

solver.solve(instance)
# for c in instance.component_objects(pyo.Param, active=True):
#     print(f"Constraint: {c.name}")
#     print(c.display())

print(instance.L.display())
print(instance.S.display())
print(instance.nS.display())
print(instance.P.extract_values()) # type: ignore


L : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :    3 : {0, 1, 2}
None
S : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :      L :    1 :    {0,}
None
nS : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :    2 : {1, 2}
None
{(0, 1, 0): 10.0, (0, 0, 1): 0.0, (1, 2, 1): 10.0, (1, 1, 2): 25.0, (2, 3, 2): 10.0, (2, 2, 3): 15.0}


In [24]:
import pyomo.kernel as pmo

m = pmo.block()
m.idx  = list(instance.LF)


m.P = pmo.variable_dict()
m.Q = pmo.variable_dict()
m.I = pmo.variable_dict()
m.U = pmo.parameter_dict()

for l, i, j in m.idx:
    m.P[l, i, j] = pmo.variable(domain=pyo.NonNegativeReals)
    m.Q[l, i, j] = pmo.variable(domain=pyo.NonNegativeReals)
    m.I[l, i, j] = pmo.variable(domain=pyo.NonNegativeReals)
    m.U[l, i, j] = pmo.parameter(3+5*i)

In [35]:
def get_vars_results(key: list, var_list: list[Union[pmo.variable_dict, pmo.parameter_dict]]):
    results = []
    
    for i in key:
        results.append(dict(map(lambda var: (var.name, var[i]()) if i in var.keys() else (var.name, None), var_list)))
    return pl.from_dicts(results).select(pl.Series(key).alias("idx"), pl.all())

get_vars_results(m.idx, [m.U])


idx,U
list[i64],i64
"[0, 1, 0]",8
"[0, 0, 1]",3
"[1, 2, 1]",13
"[1, 1, 2]",8
"[2, 3, 2]",18
"[2, 2, 3]",13
